In [1]:
import numpy as np
import torch
import torch.nn as nn
import QuantLib as ql
import time
import pickle

import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from IPython import display

import HestonUtils
from HestonNN import Net

%matplotlib inline
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = [15, 10]

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('Running on GPU')
else:
    device = torch.device('cpu')
    print('Running on CPU')

Running on GPU


Loss functions that we want to minimize the 5 Heston parameters over:

In [51]:
def nn_loss(x, observed_params, y_trues, net):
    n = len(y_trues)
    xs = np.array(list(x)*n).reshape(-1,5)
    nn_params = np.hstack((observed_params[:,:2], xs, observed_params[:,2:]))
    moneyness = nn_params[:,0]
    nn_params = torch.Tensor(nn_params).float().to(device)
    with torch.no_grad():
        y_preds = net(nn_params).to('cpu').detach().numpy().flatten()
    return np.mean((1/moneyness)**2*np.square(y_preds - y_trues))

def ql_loss(x, observed_params, y_trues):
    n = len(y_trues)
    xs = np.array(list(x)*n).reshape(-1,5)
    nn_params = np.hstack((observed_params[:,:2], xs, observed_params[:,2:]))
    moneyness = nn_params[:,0]
    ql_params = HestonUtils.convertNNtoQLparams(nn_params)
    y_preds = np.array([HestonUtils.QuantlibHestonPrice(*ql_params[i]) for i in range(n)])
    return np.mean((1/moneyness)**2*np.square(y_trues - y_preds))

In [36]:
def nn_calibration(net, observed_params, observed_price, bounds):
    res = differential_evolution(nn_loss, bounds=bounds, args=(observed_params, observed_price, net), 
                                 popsize=10, constraints=HestonUtils.feller_con, polish=False)
    return res

def ql_calibration(observed_params, observed_price, bounds):
    res = differential_evolution(ql_loss, bounds=bounds, args=(observed_params, observed_price), 
                                 popsize=10, constraints=HestonUtils.feller_con, polish=False, maxiter=100)
    return res

In [4]:
narrow_net = torch.load('models/run_133')
wide_net = torch.load('models/run_134')

In [5]:
def calibrationComparison(net=None, data_type='NARROW', N=100, M=30, seed=0):
    np.random.seed(seed)
    
    if data_type == 'NARROW':
        bounds = ((0,0.5), (0,3), (0,0.5), (0,1), (-0.9,0))
    else:
        bounds = ((0,1), (0,10), (0,1), (0,2), (-1,0))
    bounds = np.array(bounds)
    
    x_stars = np.zeros((N,5))
    x_cals_nn = np.zeros((N,5))
    fevs_nn = []
    funs_nn = [] 
    times_nn = []
    
    x_cals_ql = np.zeros((N,5))
    fevs_ql = []
    funs_ql = []
    times_ql = []
    for i in range(N):
        x_star = np.random.uniform(bounds[:,0], bounds[:,1])
        while 2*x_star[1]*x_star[2] <= x_star[3]**2:
            x_star = np.random.uniform(bounds[:,0], bounds[:,1])
        x_stars[i] = x_star
        Ss = np.random.uniform(low=0.5,high=1.5,size=M)
        Ks = np.array([1.]*M)
        moneyness = Ss / Ks
        current_dates = np.array([ql.Date(1,1,2019)]*M)
        Ts = np.random.randint(low=1,high=365*2+1,size=M)
        maturity_dates = current_dates + Ts
        Ts = Ts / 365
        rs = np.array([0.0]*M) + np.random.uniform(low=0,high=0.05)
        qs = np.array([0.0]*M) + np.random.uniform(low=0,high=0.05)
        
        observed_params = np.array([moneyness, Ts, rs, qs]).T
        
        y_trues = np.array([HestonUtils.QuantlibHestonPrice(Ss[i], Ks[i], current_dates[i], maturity_dates[i],
                                                            *x_star, rs[i], qs[i]) for i in range(M)])
        
        time_start = time.time()
        x_cal_nn = nn_calibration(net, observed_params, y_trues, bounds)
        times_nn.append(time.time()-time_start)
        
        time_start = time.time()
        x_cal_ql = ql_calibration(observed_params, y_trues, bounds)
        times_ql.append(time.time()-time_start)
        
        x_cals_nn[i] = x_cal_nn.x
        fevs_nn.append(x_cal_nn.nfev)
        funs_nn.append(x_cal_nn.fun)
        
        x_cals_ql[i] = x_cal_ql.x
        fevs_ql.append(x_cal_ql.nfev)
        funs_ql.append(x_cal_ql.fun)
    fevs_nn = np.array(fevs_nn).reshape(-1,1)
    funs_nn = np.array(funs_nn).reshape(-1,1)
    times_nn = np.array(times_nn).reshape(-1,1)
    fevs_ql = np.array(fevs_ql).reshape(-1,1)
    funs_ql = np.array(funs_ql).reshape(-1,1)
    times_ql = np.array(times_ql).reshape(-1,1)
    return x_stars, np.hstack((x_cals_nn, funs_nn, fevs_nn, times_nn)), np.hstack((x_cals_ql, funs_ql, fevs_ql, times_ql))

In [6]:
# narrow = calibrationComparison(net=narrow_net, data_type='NARROW', N=200, M=30)

In [37]:
def calibrationAnalysis(net=None, data_type='NARROW', N=100, M=30, seed=0):
    np.random.seed(seed)
    
    if data_type == 'NARROW':
        bounds = ((0,0.5), (0,3), (0,0.5), (0,1), (-0.9,0))
    else:
        bounds = ((0,1), (0,10), (0,1), (0,2), (-1,0))
    bounds = np.array(bounds)
    
    x_stars = np.zeros((N,5))
    x_cals_nn = np.zeros((N,5))
    funs_nn = [] 
    times_nn = []
    for i in range(N):
        x_star = np.random.uniform(bounds[:,0], bounds[:,1])
        while 2*x_star[1]*x_star[2] <= x_star[3]**2:
            x_star = np.random.uniform(bounds[:,0], bounds[:,1])
        x_stars[i] = x_star
        Ss = np.random.uniform(low=0.5,high=1.5,size=M)
        Ks = np.array([1.]*M)
        moneyness = Ss / Ks
        current_dates = np.array([ql.Date(1,1,2019)]*M)
        Ts = np.random.randint(low=1,high=365*2+1,size=M)
        maturity_dates = current_dates + Ts
        Ts = Ts / 365
        rs = np.array([0.0]*M) + np.random.uniform(low=0,high=0.05)
        qs = np.array([0.0]*M) + np.random.uniform(low=0,high=0.05)
        
        observed_params = np.array([moneyness, Ts, rs, qs]).T
        
        y_trues = np.array([HestonUtils.QuantlibHestonPrice(Ss[i], Ks[i], current_dates[i], maturity_dates[i],
                                                            *x_star, rs[i], qs[i]) for i in range(M)])
        
        time_start = time.time()
        x_cal_nn = nn_calibration(net, observed_params, y_trues, bounds)
        times_nn.append(time.time()-time_start)
        
        x_cals_nn[i] = x_cal_nn.x
        funs_nn.append(x_cal_nn.fun)
    funs_nn = np.array(funs_nn).reshape(-1,1)
    times_nn = np.array(times_nn).reshape(-1,1)
    return x_stars, np.hstack((x_cals_nn, funs_nn, times_nn))


In [38]:
analysis_narrow = calibrationAnalysis(net=narrow_net, data_type='NARROW', N=1000, M=30, seed=1)

In [50]:
a0 = np.load('results/calibration/narrow0.npy')
a1 = np.load('results/calibration/narrow1.npy')
a2 = np.load('results/calibration/narrow2.npy')
a3 = np.load('results/calibration/narrow3.npy')

error0 = a0 - a1[:,:5]
error1 = a2 - a3[:,:5]

print(np.mean(abs(error0), axis=0))
print(np.mean(abs(error1), axis=0))


[0.0002156  0.01917351 0.00196022 0.00734577 0.03071672]
[0.00022647 0.02046394 0.00189385 0.00880682 0.03717033]


In [161]:
def nn_loss_(x, observed_params, y_trues, net):  
    n = len(y_trues)
    xs = x.repeat(n).reshape(-1,5)
    nn_params = torch.cat((observed_params[:,:2],xs, observed_params[:,2:]),axis=1).to(device)
    moneyness = nn_params[:,0]
    y_preds = net(nn_params).flatten()
    error = torch.mean((1/moneyness)**2*(y_preds - y_trues)**2)
    return error

In [168]:
def calibrationSensitivity(net=None, data_type='NARROW', N=100, M=30, seed=0):
    np.random.seed(seed)
    
    if data_type == 'NARROW':
        bounds = ((0,0.5), (0,3), (0,0.5), (0,1), (-0.9,0))
    else:
        bounds = ((0,1), (0,10), (0,1), (0,2), (-1,0))
    bounds = np.array(bounds)
    
    x_stars = np.zeros((N,5))
    x_cals_nn = np.zeros((N,5))
    funs_nn = [] 
    times_nn = []
    grads = []
    for i in range(N):
        x_star = np.random.uniform(bounds[:,0], bounds[:,1])
        while 2*x_star[1]*x_star[2] <= x_star[3]**2:
            x_star = np.random.uniform(bounds[:,0], bounds[:,1])
        x_stars[i] = x_star
        Ss = np.random.uniform(low=0.5,high=1.5,size=M)
        Ks = np.array([1.]*M)
        moneyness = Ss / Ks
        current_dates = np.array([ql.Date(1,1,2019)]*M)
        Ts = np.random.randint(low=1,high=365*2+1,size=M)
        maturity_dates = current_dates + Ts
        Ts = Ts / 365
        rs = np.array([0.0]*M) + np.random.uniform(low=0,high=0.05)
        qs = np.array([0.0]*M) + np.random.uniform(low=0,high=0.05)
        
        observed_params = np.array([moneyness, Ts, rs, qs]).T
        
        y_trues = np.array([HestonUtils.QuantlibHestonPrice(Ss[i], Ks[i], current_dates[i], maturity_dates[i],
                                                            *x_star, rs[i], qs[i]) for i in range(M)])
        
        x_star_ = torch.Tensor(x_star).to(device)
        observed_params_ = torch.Tensor(observed_params).to(device)
        y_trues_ = torch.Tensor(y_trues).to(device)
        x_star_.requires_grad = True
        asdf = nn_loss_(x_star_, observed_params_, y_trues_, net)
        grad_params = torch.autograd.grad(asdf, x_star_, create_graph=True)[0]
        fdsa = np.zeros((5,5))
        for j in range(5):
            fdsa[j] = torch.autograd.grad(grad_params[j], x_star_, create_graph=True)[0].to('cpu').detach().numpy()
        grads.append(np.diagonal(fdsa))
        
        time_start = time.time()
        x_cal_nn = nn_calibration(net, observed_params, y_trues, bounds)
        times_nn.append(time.time()-time_start)
        
        x_cals_nn[i] = x_cal_nn.x
        funs_nn.append(x_cal_nn.fun)
    funs_nn = np.array(funs_nn).reshape(-1,1)
    times_nn = np.array(times_nn).reshape(-1,1)
#     return x_stars, np.hstack((x_cals_nn, funs_nn, times_nn))
    return grads


In [172]:
test = calibrationSensitivity(net=narrow_net, data_type='NARROW', N=1000, M=30, seed=1234)